In [ ]:
# agent_router_with_memory.py

import os
import re
import numpy as np
from typing import Any, Dict, List, TypedDict

from langchain import LLMChain
from langchain.llms import HuggingFacePipeline
from langchain.schema import BaseMemory
from langchain.memory import ConversationBufferMemory, CombinedMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import Tool, AgentType, create_pandas_dataframe_agent

from deep_translator import GoogleTranslator
from sklearn.metrics.pairwise import cosine_similarity
from langgraph.graph import StateGraph, START, END

from sentence_transformers import SentenceTransformer
from transformers import pipeline, LlamaForCausalLM, LlamaTokenizer

# ——————————————————————————————————————————————————————————————
# 1) Llama 3.2-3B local HF loading
# ——————————————————————————————————————————————————————————————

MODEL_PATH = "/path/to/your/llama-3.2-3b-checkpoint"
# force local-only so it won’t recurse into unwanted subfolders:
tokenizer = LlamaTokenizer.from_pretrained(
    MODEL_PATH, trust_remote_code=True, local_files_only=True
)
model = LlamaForCausalLM.from_pretrained(
    MODEL_PATH, 
    device_map="auto", 
    torch_dtype="auto", 
    trust_remote_code=True,
    local_files_only=True,
)

text_gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.2,
    top_p=0.9,
    device_map="auto",
)

hf_llm = HuggingFacePipeline(pipeline=text_gen)


# ——————————————————————————————————————————————————————————————
# 2) Define your 3 tools
# ——————————————————————————————————————————————————————————————

def calculator_tool(expr: str) -> str:
    try:
        return f"The result of '{expr}' is {eval(expr)}"
    except Exception as e:
        return f"Error: {e}"
calculator = Tool("Calculator", calculator_tool, "Basic arithmetic")

def sentiment_analysis_tool(text: str) -> str:
    t = text.lower()
    if any(w in t for w in ("happy","great","good","excellent")):
        return "Positive 👍"
    if any(w in t for w in ("sad","bad","terrible","awful")):
        return "Negative 👎"
    return "Neutral 🤔"
sentiment = Tool("SentimentAnalysis", sentiment_analysis_tool, "Detect sentiment")

# Pandas agent: uses the same hf_llm under the hood
# assume `df` is defined elsewhere in your script
pandas_agent = create_pandas_dataframe_agent(
    hf_llm,
    df,
    verbose=True,
    allow_dangerous_code=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)
def run_pandas(q: str) -> str:
    return pandas_agent.run(q)
pandas = Tool("PandasDataAnalysis", run_pandas, "Query Pandas DataFrame")


# ——————————————————————————————————————————————————————————————
# 3) Episodic Memory & Semantic Cache
# ——————————————————————————————————————————————————————————————

_long_term: Dict[str,List[str]]    = {}
_qa_cache:    Dict[str,str]        = {}
_qa_embeds:   Dict[str,np.ndarray] = {}
SESSION_ID = "user_123"

def update_long_term_memory(user: str, inp: str, outp: str):
    mem = _long_term.setdefault(user, [])
    if inp:  mem.append(f"User: {inp}")
    if outp: mem.append(f"Bot:  {outp}")
    _long_term[user] = mem[-10:]

def get_long_term_memory(user: str) -> str:
    return "\n".join(_long_term.get(user, []))


# load sentence-transformers locally, no recursion
EMB_MODEL = SentenceTransformer(
    "/path/to/your/all-MiniLM-L6-v2", 
    local_files_only=True
)

def find_similar_cached(q: str, threshold: float = 0.85) -> str:
    vec = EMB_MODEL.encode(q, convert_to_numpy=True)
    for orig, ov in _qa_embeds.items():
        sim = cosine_similarity(vec.reshape(1,-1), ov.reshape(1,-1))[0][0]
        if sim >= threshold:
            return orig
    return None


TOOL_KEYWORDS = ["calculate","sentiment","pandas","data"]
def is_tool_query(txt: str) -> bool:
    low = txt.lower().strip()
    return any(low.startswith(kw) for kw in TOOL_KEYWORDS)


# ——————————————————————————————————————————————————————————————
# 4) LangGraph router for those 3 tools
# ——————————————————————————————————————————————————————————————

class RouterState(TypedDict):
    input: str
    decision: str
    output: Any

def llm_call_router(state: RouterState) -> RouterState:
    t = state["input"].strip().lower()
    if t.startswith("calculate"):
        state["decision"] = "Calculator"
    elif any(w in t for w in ("happy","sad","sentiment")):
        state["decision"] = "SentimentAnalysis"
    else:
        state["decision"] = "PandasDataAnalysis"
    return state

def run_Calculator(s):    s["output"]=calculator_tool(s["input"]); return s
def run_Sentiment(s):     s["output"]=sentiment_analysis_tool(s["input"]); return s
def run_Pandas(s):        s["output"]=run_pandas(s["input"]);            return s

router = StateGraph(state_schema=RouterState)
router.add_node("router", llm_call_router)
router.add_node("Calculator", run_Calculator)
router.add_node("SentimentAnalysis", run_Sentiment)
router.add_node("PandasDataAnalysis", run_Pandas)

router.add_edge(START, "router")
router.add_conditional_edges("router", lambda s: s["decision"], {
    "Calculator":"Calculator",
    "SentimentAnalysis":"SentimentAnalysis",
    "PandasDataAnalysis":"PandasDataAnalysis"
})
for node in ("Calculator","SentimentAnalysis","PandasDataAnalysis"):
    router.add_edge(node, END)

router.set_entry_point("router")
router_workflow = router.compile()


# ——————————————————————————————————————————————————————————————
# 5) Pure‐chat Llama chain + memory
# ——————————————————————————————————————————————————————————————

prompt = ChatPromptTemplate.from_messages([
    ("system","You are a helpful AI assistant."),
    ("system","Chat history:\n{chat_history}"),
    ("system","Long-term memory:\n{long_term_memory}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human","{input}"),
])

short_term = ConversationBufferMemory(
    memory_key="chat_history",
    input_key="input",
    return_messages=True
)
long_term = CombinedMemory([short_term, LongTermChatMemory(session_id=SESSION_ID)])

chat_chain = LLMChain(
    prompt=prompt,
    llm=hf_llm,
    memory=long_term
)


# ——————————————————————————————————————————————————————————————
# 6) Unified converse()
# ——————————————————————————————————————————————————————————————

def converse(user_input: str) -> str:
    low = user_input.lower().strip()
    if "remember" in low:
        return "Memory:\n" + get_long_term_memory(SESSION_ID)

    if is_tool_query(user_input):
        # semantic cache
        sim = find_similar_cached(user_input)
        if sim:
            return _qa_cache[sim]
        st  = router_workflow.invoke({"input": user_input})
        out = st["output"]
        _qa_cache[user_input]    = out
        _qa_embeds[user_input]   = EMB_MODEL.encode(user_input, convert_to_numpy=True)
        update_long_term_memory(SESSION_ID, user_input, out)
        return out

    # fallback to pure chat
    out = chat_chain.run(user_input)
    update_long_term_memory(SESSION_ID, user_input, out)
    return out


# ——————————————————————————————————————————————————————————————
# 7) Smoke‐test
# ——————————————————————————————————————————————————————————————

if __name__ == "__main__":
    for q in [
        "Calculate 3 * 8",
        "How am I feeling? Sentiment 'I am sad'",
        "Show me summary stats on the DataFrame",
        "Do you remember our previous requests?"
    ]:
        print("→", converse(q))


In [ ]:
# agent_router_llm_driven.py

import os
import re
from typing import Any, Dict, TypedDict

from langchain import LLMChain
from langchain.llms import HuggingFacePipeline
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import Tool
from langchain.schema import BaseMemory  # still required by LLMChain signature

from transformers import pipeline, LlamaForCausalLM, LlamaTokenizer

from langgraph.graph import StateGraph, START, END

# ——————————————————————————————————————————————————————————————
# 1) Spin up your local Llama 3.2-3B via HF pipeline
# ——————————————————————————————————————————————————————————————

MODEL_PATH = "/path/to/your/llama-3.2-3b-checkpoint/"

tokenizer = LlamaTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)
model     = LlamaForCausalLM.from_pretrained(
    MODEL_PATH, device_map="auto", torch_dtype="auto", trust_remote_code=True
)

text_gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.2,
    top_p=0.9,
    device_map="auto",
)
hf_llm = HuggingFacePipeline(pipeline=text_gen)


# ——————————————————————————————————————————————————————————————
# 2) Define your two real tools
# ——————————————————————————————————————————————————————————————

def calculator_tool(expr: str) -> str:
    try:
        return f"The result of '{expr}' is {eval(expr)}."
    except Exception as e:
        return f"Error: {e}"
calculator = Tool("Calculator", calculator_tool, "Basic arithmetic")

def sentiment_analysis_tool(text: str) -> str:
    low = text.lower()
    if any(w in low for w in ("happy","great","good","excellent")):
        return "Positive 👍"
    if any(w in low for w in ("sad","bad","terrible","awful")):
        return "Negative 👎"
    return "Neutral 🤔"
sentiment = Tool("SentimentAnalysis", sentiment_analysis_tool, "Detect sentiment")


# ——————————————————————————————————————————————————————————————
# 3) Stateless Chat chain (our fallback “tool”)
# ——————————————————————————————————————————————————————————————

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant."),
    ("system", "If nothing else, just answer conversationally."),
    MessagesPlaceholder(variable_name="chat_history"),  # will always be empty here
    ("human", "{input}"),
])
chat_chain = LLMChain(prompt=chat_prompt, llm=hf_llm)


# ——————————————————————————————————————————————————————————————
# 4) LLM-driven router chain
# ——————————————————————————————————————————————————————————————

router_prompt = ChatPromptTemplate.from_messages([
    ("system", 
     "You are a routing assistant.  Given a user query, "
     "choose exactly one of: Calculator, SentimentAnalysis, Chat.  "
     "Respond with only that tool name."),
    ("user", "{input}"),
])
router_chain = LLMChain(prompt=router_prompt, llm=hf_llm)


# ——————————————————————————————————————————————————————————————
# 5) LangGraph router wiring
# ——————————————————————————————————————————————————————————————

class RouterState(TypedDict):
    input: str
    decision: str
    output: Any

def llm_call_router(state: RouterState) -> RouterState:
    # ask our LLM which tool to use
    tool_name = router_chain.run(input=state["input"]).strip()
    # safe-guard: default to Chat if unexpected
    if tool_name not in ("Calculator", "SentimentAnalysis", "Chat"):
        tool_name = "Chat"
    state["decision"] = tool_name
    return state

def run_Calculator(state: RouterState) -> RouterState:
    # strip leading command if present
    expr = re.sub(r'^(calculate|calc)\s+', '', state["input"], flags=re.IGNORECASE)
    state["output"] = calculator_tool(expr)
    return state

def run_SentimentAnalysis(state: RouterState) -> RouterState:
    state["output"] = sentiment_analysis_tool(state["input"])
    return state

def run_Chat(state: RouterState) -> RouterState:
    state["output"] = chat_chain.run(input=state["input"])
    return state

router = StateGraph(state_schema=RouterState)
router.add_node("router", llm_call_router)
router.add_node("Calculator", run_Calculator)
router.add_node("SentimentAnalysis", run_SentimentAnalysis)
router.add_node("Chat", run_Chat)

router.add_edge(START, "router")
router.add_conditional_edges(
    "router",
    lambda s: s["decision"],
    {
        "Calculator": "Calculator",
        "SentimentAnalysis": "SentimentAnalysis",
        "Chat": "Chat"
    }
)
for node in ("Calculator", "SentimentAnalysis", "Chat"):
    router.add_edge(node, END)

router.set_entry_point("router")
router_workflow = router.compile()


# ——————————————————————————————————————————————————————————————
# 6) Unified entrypoint
# ——————————————————————————————————————————————————————————————

def converse(user_input: str) -> str:
    # route every query
    st = router_workflow.invoke({"input": user_input})
    return st["output"]


# ——————————————————————————————————————————————————————————————
# 7) Quick smoke-test
# ——————————————————————————————————————————————————————————————

if __name__ == "__main__":
    for q in [
        "Calculate 3 * 8",
        "How am I feeling? I am sad today.",
        "What is the capital of France?",
        "Tell me a joke"
    ]:
        print(f">>> {q!r}")
        print("→", converse(q), "\n")
